In [1]:
import pandas as pd
import numpy as np
import ast

pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames


In [2]:
df = pd.read_csv("files/credits.csv")

df.head(2)

# queremos extraer la informacion de crew (para tener datos de direccion, guionistas y restos de trabajadores de la industria)
# y mantener la asociación con el id (que es el id de cada película)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844


In [3]:
df['crew'][0]
# vemos que el contenido de cada fila es un string, que contiene una lista con diccionarios. 

'[{\'credit_id\': \'52fe4284c3a36847f8024f49\', \'department\': \'Directing\', \'gender\': 2, \'id\': 7879, \'job\': \'Director\', \'name\': \'John Lasseter\', \'profile_path\': \'/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f4f\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12891, \'job\': \'Screenplay\', \'name\': \'Joss Whedon\', \'profile_path\': \'/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f55\', \'department\': \'Writing\', \'gender\': 2, \'id\': 7, \'job\': \'Screenplay\', \'name\': \'Andrew Stanton\', \'profile_path\': \'/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f5b\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12892, \'job\': \'Screenplay\', \'name\': \'Joel Cohen\', \'profile_path\': \'/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f61\', \'department\': \'Writing\', \'gender\': 0, \'id\': 12893, \'job\': \'Screenplay\', \'name\': \'A

In [4]:
crew =  []
# vamos a crear una lista para ir añadiendo los diccionarios según extraigamos la informacion

for film_id, fila in zip(df['id'], df['crew']):
# iteramos al mismo tiempo cada fila de df['id'] con cada fila de df['crew'] para que no perdamos la asociacion entre el 
# id de la película y toda la info que hay en df['crew']

    # Convertir el string en una lista de diccionarios
    fila = ast.literal_eval(fila)
    for elemento in fila:
        elemento.update({'id': film_id})
        # cambiamos el id que estaba en crew, que iba asociado con el id del trabajador, por el id de la película, porque 
        # es lo que nos interesa, ver trabajadores para cada película. 
        crew.append(elemento)
    

In [5]:
# Convertimos la lista de diccionarios en un Dataframe, que contiene la informacion contenida en cada diccionario que 
# teniamos en df['crew'] lo único cambiando el id por el id de la película
df_crew = pd.DataFrame(crew)

In [6]:
df_crew.head()

,credit_id,department,gender,id,job,name,profile_path
0,52fe4284c3a36847f8024f49,Directing,2,862,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,52fe4284c3a36847f8024f4f,Writing,2,862,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,52fe4284c3a36847f8024f55,Writing,2,862,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,52fe4284c3a36847f8024f5b,Writing,2,862,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,52fe4284c3a36847f8024f61,Writing,0,862,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg


In [7]:
# nos quedamos con las columnas que nos interesan, que en este caso son solo name, department, job, gender y id.  
df_crew.drop(columns=['credit_id', 'profile_path'], inplace=True)
df_crew

,department,gender,id,job,name
0,Directing,2,862,Director,John Lasseter
1,Writing,2,862,Screenplay,Joss Whedon
2,Writing,2,862,Screenplay,Andrew Stanton
3,Writing,2,862,Screenplay,Joel Cohen
4,Writing,0,862,Screenplay,Alec Sokolow
...,...,...,...,...,...
464309,Sound,0,67758,Original Music Composer,Richard McHugh
464310,Camera,2,67758,Director of Photography,João Fernandes
464311,Directing,0,227506,Director,Yakov Protazanov
464312,Production,2,227506,Producer,Joseph N. Ermolieff


In [8]:
df_crew['gender'].unique()
# el 2 se corresponde con hombres, el 1 con mujeres y el 0 faltan por asignar. Para tratar de reducir al máximo los 0 y tener 
# algo más de informacion hacemos una función para poder usarla sobre el dataframe que no tiene datos de genero. 

array([2, 0, 1], dtype=int64)

In [9]:
# filtramos solo el dataframe que no tiene información de genero
df_crew_no_data = df_crew[df_crew['gender'] == 0]

In [10]:
# hacemos una lista de nombres de hombres y mujeres
female_names_extended = [
    "Mary", "Helen", "Kim", "Marilyn", "Julie", "Megan", "Jennifer", "Sarah", 
    "Lisa", "Linda", "Karen", "Laura", "Emily", "Jessica", "Amanda", "Emma", 
    "Samantha", "Susan", "Michelle", "Nancy", "Betty", "Margaret", "Dorothy", 
    'Lyndell Quiyou', 'Manon', 'Star', 'Victoria', 'Patsy', 'Daisy', 'Angi', 'Genevieve', 'Julia',
    'Alysia', ' Eleanor', 'Janine', 'Katie', 'Kathy', 'Lori', 'Ola', 'Marla', 'Alicja', 'María',
    'Desirée', 'Ada', 'Ginger', 'Kelly', 'Nathalie', 'Nelly', 'Melissa', 'Renee', 'Jo', 'Elaine', 
    'Isabel', 'Liz', 'Nina', 'Livia', 'Bonnie', 'Marcina', 'Marisa', 'Cornelia', 'Claudia', 'Vijayendra', 
    'Gioia', 'Colleen', 'Martha', 'Amy', 'Beth','Ana', 'Laura', 'Cynthis', 'Yvonne', 'Libbie', 'Lucila', 'Jen']



male_names_extended = [
    "Alec", "Ed", "John", "Robert", "Michael", "David", "James", "William", 
    "Chris", "Richard", "Charles", "Paul", "Mark", "Steven", "Andrew", "Thomas", 
    "Joshua", "Daniel", "George", "Kevin", "Brian", "Donald", "Timothy", "Jason", 
    'Greg', 'Doug', 'Ib', 'Patrik', 'Harald', 'Arthur', 'Larry', 'Gary', 'Jeff', 
    'Hal', 'Don', "Joe", "Dale", 'Lav', 'Yakov', 'Carmelo', 'Javier', 'Anders', 'Stephen', 
    'Mike', 'Genki', 'Marcello', 'Rob', 'Alex', 'Christopher', 'Rainer', 'Les', 'Ian', 'José', 
    'Ramón', 'Stewart', 'Charlie', 'Gabriel', 'Gordon', 'Tony', 'Agustín', 'Petur', 'Henry', 
    'Michel', 'Alfred', 'Craig', 'Bruce', 'Jacques', 'Marcus', 'Pascal', 'Stanley', 'Tom', 'Steffen', 'Patrice',
    'Zach', 'Peter', 'Adam', 'Bob', 'Luis', 'Ol', 'Piero', 'Kent', 'Gaël', 'Sidney', 'Jack', 'Jamie', 'Pablo' , 
    'Alan', 'Stefan', 'Ira', 'Jon', 'Justin', 'Jorge', 'Walter', 'Frank', 'Norman', 'Nate', 'Nicholas', 'Joel', 'Marc', 
    'Oscar', 'Hugo', 'Joseph', 'Barry', 'Thom', 'Brice', 'Patrick', 'Lester',
    'Dan', 'Horace', 'Gerald', 'Neil', 'Eliot', 'Otto', 'Juan', 'Matt', 'Bryan', 'Satoshi', 'Paulo', 'Guy']

def assign_gender_extended(name):
    # accedemos al first name de df_crew_no_data['name'], para ello hacemos un split y nos quedamos con el primer elemento. 
    # quitamos espacios de delante y de detrás con método strip()
    
    first_name = name.split()[0].strip()
    
    # Comprobamos si el nombre está en la lista de hombres y mujeres, en caso de estar en una u otra asignamos 1 si es mujer o
    # 2 si es hombre, si no está el nombre asigna 0
    if first_name in female_names_extended:
        return 1  # Female
    elif first_name in male_names_extended:
        return 2  # Male
    else:
        return 0  # Unknown

# Usamos el metodo apply de pandas para poder usar esta función en nuestro Dataframe. 

df_crew_no_data['gender'] = df_crew_no_data['name'].apply(assign_gender_extended)



C:\Users\Laura Romero\AppData\Local\Temp\ipykernel_9220\2888040516.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_crew_no_data['gender'] = df_crew_no_data['name'].apply(assign_gender_extended)


In [11]:
df_crew[df_crew['gender'] == 0] = df_crew_no_data
# cambiamos las columnas de df_crew donde el genero es 0 por las nuevas columnas que hemos creado en df_crew_no_data


In [12]:
df_crew['department'].value_counts()

department
Production           94498
Writing              74831
Directing            58134
Sound                50605
Art                  40694
Camera               33539
Crew                 31605
Costume & Make-Up    30850
Editing              29831
Visual Effects       14861
Lighting              4847
Actors                  19
Name: count, dtype: int64

In [13]:
# filtramos para quedarnos unicamente con aquellos departamentos que nos interesan, descartando Sound, Camera, Visual Effects, Art. 

In [14]:
df_crew = df_crew[(df_crew['department'] != 'Sound')&(df_crew['department'] != 'Camera')&(df_crew['department'] != 'Art')&(df_crew['department'] != 'Visual Effects')&(df_crew['department'] != 'Lighting')&(df_crew['department'] != 'Editing')]

In [15]:
df_crew['department'].value_counts()

department
Production           94498
Writing              74831
Directing            58134
Crew                 31605
Costume & Make-Up    30850
Actors                  19
Name: count, dtype: int64

In [18]:
df_crew = df_crew[df_crew['gender'] != 0]
df_crew.drop_duplicates(inplace=True)

In [19]:

df_crew.to_csv('files/crew_no_0.csv', index=False)